In [1]:
!pip install roboflow  ultralytics opencv-python-headless pandas -q

import cv2, math, time, csv
import numpy as np
import pandas as pd
from collections import deque, defaultdict
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

In [2]:
# Colab setup (optional)
import sys, os
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    # Ensure we work under /content for predictable paths
    try:
        from google.colab import drive  # noqa
        # Commented out by default; uncomment if you want to use Drive
        # drive.mount('/content/drive')
        pass
    except Exception as e:
        print("Colab Drive not available or not needed:", e)
    print("Running in Colab. Working under /content.")
else:
    print("Not running in Colab. Paths may differ.")


Running in Colab. Working under /content.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Download dataset from Roboflow (YOLOv11 format)
!pip install roboflow -q

from roboflow import Roboflow
rf = Roboflow(api_key="yh12edAfMgslAlFwL3Ip")
project = rf.workspace("shihab-sarar-936jx").project("threat-detection-x42o1")
version = project.version(1)
dataset = version.download("yolov11")

# Build DATA_YAML path from the downloaded dataset
from pathlib import Path
DATA_DIR = Path(getattr(dataset, "location", "/content"))
DATA_YAML = str(DATA_DIR / "data.yaml")
print("Dataset downloaded to:", DATA_DIR)
print("DATA_YAML:", DATA_YAML)


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to threat-detection-1 in yolov11:: 100%|██████████| 417/417 [00:00<00:00, 2066.67it/s]

Dataset downloaded to: /content/threat-detection-1
DATA_YAML: /content/threat-detection-1/data.yaml


In [7]:
# Install PyTorch CUDA build (typical for Colab Free)
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Quick check
import torch, platform
print("torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Looking in indexes: https://download.pytorch.org/whl/cu121
torch: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


In [12]:
# Uninstall PyTorch and related packages
!pip uninstall -y torch torchvision torchaudio xformers

# (Optional) Clear pip cache to free space
!pip cache purge -q



Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124


In [14]:

DATA_YAML = DATA_YAML
EPOCHS = 100
MODEL_BASE = "yolo11n.pt"

model = YOLO(MODEL_BASE)
model.train(data=DATA_YAML, epochs=EPOCHS, imgsz=768)


Ultralytics 8.3.189 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/threat-detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ca8021beb50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [16]:
from ultralytics import YOLO
import cv2

model = YOLO("/content/runs/detect/train6/weights/best.pt")

cap = cv2.VideoCapture("/content/10.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection
    results = model.predict(frame, conf=0.5)

    # Plot results
    annotated_frame = results[0].plot()

    # Save to output video
    out.write(annotated_frame)

cap.release()
out.release()



0: 448x768 1 fence, 1 person, 59.3ms
Speed: 4.2ms preprocess, 59.3ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 15.9ms
Speed: 4.7ms preprocess, 15.9ms inference, 1.9ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 28.3ms
Speed: 4.7ms preprocess, 28.3ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 16.8ms
Speed: 6.9ms preprocess, 16.8ms inference, 3.6ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 14.3ms
Speed: 4.5ms preprocess, 14.3ms inference, 2.3ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 15.2ms
Speed: 4.9ms preprocess, 15.2ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 13.4ms
Speed: 4.8ms preprocess, 13.4ms inference, 2.1ms postprocess per image at shape (1, 3, 448, 768)

0: 448x768 1 fence, 1 person, 19.5ms
Speed: 4.5ms prep

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 4.0s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


video 1/1 (frame 1/304) /kaggle/input/video16/16.mp4: 384x640 3 persons, 63.9ms
video 1/1 (frame 2/304) /kaggle/input/video16/16.mp4: 384x640 3 persons, 40.6ms
video 1/1 (frame 3/304) /kaggle/input/video16/16.mp4: 384x640 1 person, 40.7ms
video 1/1 (frame 4/304) /kaggle/input/video16/16.mp4: 384x640 2 persons, 45.9ms
video 1/1 (frame 5/304) /kaggle/input/video16/16.mp4: 384x640 1 person, 42.9ms
video 1/1 (frame 6/304) /kaggle/input/video16/16.mp4: 384x640 3 persons, 43.5ms
video 1/1 (frame 7/304) /kaggle/input/video16/16.mp4: 384x640 2 persons, 41.4ms
video 1/1 (frame 8/304) /kaggle/input/video16/16.mp4: 384x640 1 fence, 2 persons, 42.5ms
video 1/1 (frame 9/304) /kaggle/input/video16/16.mp4: 384x640 1 fence, 2 persons, 42.1ms
video 1/1 (frame 10/304) /kaggle/input/video16/1

In [ ]:
import pandas as pd

df=pd.read_csv("/content/alerts.csv")


In [ ]:
df

,time,frame,track_id,alert,x,y,distance_px


### Notes (Colab)
- Outputs like `annotated.mp4` and `alerts.csv` are saved under `/content/`.
- If you prefer Google Drive persistence, uncomment the `drive.mount` line in the setup cell and change paths to your Drive (e.g., `/content/drive/MyDrive/...`).
